In [2]:
import os
import dspy
import json

In [3]:
DATA_TO_VALIDATE_PATH = os.path.join(os.getcwd(), 'processed', 'data.json')

In [4]:
# Read the dataset.
with open(DATA_TO_VALIDATE_PATH, 'r') as f:
    dataset = json.load(f)

In [5]:
dataset.keys()

dict_keys(['queries', 'answers', 'corpus', 'relevant_docs'])

In [6]:
# Print an example from each key of dataset
for key in dataset.keys():
    print(f"{key}:")
    for k,v in dataset[key].items():
        print(f"\t{k}: {v}")
        break
    print()

queries:
	dbf7056c-44bf-4872-bfa3-cc4cc9f405e6: What was the impact of oropharyngeal anesthesia on obstructive sleep apnea in the study subjects?

answers:
	dbf7056c-44bf-4872-bfa3-cc4cc9f405e6: Oropharyngeal anesthesia led to an increase in obstructive apneas and hypopneas, movement arousals, and oxyhemoglobin desaturations compared to the control night.

corpus:
	0: Obstructive sleep apnea following topical oropharyngeal anesthesia in loud snorers. Previous studies support the presence of an upper airway reflex mechanism that contributes to the maintenance of upper airway patency during sleep. We investigated the possibility that interference with this reflex mechanism contributes to the development of obstructive sleep apnea. Eight otherwise asymptomatic snorers (seven male and one female), age 39 +/- 5.3 yr (mean +/- SEM), underwent overnight sleep studies on three successive nights. An acclimatization night was followed by two study nights randomly assigned to control (C) and orop

## RAGAS STUFF STARTS HERE

In [7]:
# !pip install ragas

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
   ---------------------------------------- 69.7/69.7 kB ?

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy-ai 2.3.4 requires joblib~=1.3.2, but you have joblib 1.1.1 which is incompatible.


In [8]:
import pandas as pd
# Creating the DataFrame
data = []
for search_term_id, search_term_text in dataset['search_terms'].items():
    response_text = dataset['responses'].get(search_term_id)
    doc_ids = dataset['pertinent_documents'].get(search_term_id, [])
    for doc_id in doc_ids:
        data_bank_text = dataset['data_bank'].get(doc_id)
        data.append({"question": search_term_text, "ground_truths": [response_text], "answer": response_text, "contexts": [data_bank_text]})

df = pd.DataFrame(data)
df.head()

,question,ground_truths,answer,contexts
0,What was the impact of oropharyngeal anesthesi...,[Oropharyngeal anesthesia led to an increase i...,Oropharyngeal anesthesia led to an increase in...,[Obstructive sleep apnea following topical oro...
1,What was the treatment that resulted in clinic...,[The patient was treated with 150 mg of cyclop...,The patient was treated with 150 mg of cycloph...,[Paraneoplastic vasculitic neuropathy: a treat...
2,What were the beneficial responses observed in...,[The beneficial responses included regression ...,The beneficial responses included regression o...,[Treatment of childhood angiomatous diseases w...
3,Which cells in Warthin's tumor exhibit immunor...,[],,[Expression of major histocompatibility comple...
4,What was the conclusion regarding the role of ...,[The conclusion was that CNS prophylaxis with ...,The conclusion was that CNS prophylaxis with r...,[Questionable role of CNS radioprophylaxis in ...


In [9]:
from datasets import Dataset
ds = Dataset.from_pandas(df)

In [12]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_similarity,
    context_relevancy
)

result = evaluate(
    ds,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_similarity,
        context_relevancy
    ],
)

c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Evaluating:   0%|          | 0/11310 [00:00<?, ?it/s]

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\ragas\executor.py", line 93, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\ragas\executor.py", line 81, in _aresults
    raise e
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\ragas\executor.py", line 76, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "c:\Users\Hem Chandra\AppData\Local\Programs\Python\P

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
result.to_pandas().head()